In [106]:
import pyomo.environ as pyo
import pandas as pd
from itertools import combinations
import numpy as np

landa = 16
mu = 5

model = pyo.ConcreteModel()

prosumers1 = pd.read_excel("C:/Users/user/Desktop/pro.xlsx", header=0, index_col=0)
R = list(prosumers1.index)
C = list(prosumers1.columns)
p = {(r, c): prosumers1.at[r, c] for r in R for c in C}
print(p)

{(1, 't1'): 0.6, (1, 't2'): 0.1, (1, 't3'): -1.0, (1, 't4'): 0.4, (2, 't1'): 1.6, (2, 't2'): 0.7, (2, 't3'): 1.1, (2, 't4'): 1.3, (3, 't1'): 0.5, (3, 't2'): 0.8, (3, 't3'): 1.2, (3, 't4'): 1.5}


In [107]:
N = R.copy()
delta = []
for r in range(1, len(N) + 1):
    for comb in combinations(N, r):
        delta.append(list(comb))
for i in range(len(delta)):
    globals()["delta{}".format(i + 1)] = delta[i]
print(delta[3])

[1, 2]


In [108]:
# set
model.t = pyo.Set(initialize=C)
model.n = pyo.Set(initialize=N)

# parameters

model.p = pyo.Param(model.n, model.t, initialize=p)

In [109]:
#variables
model.epsilon = pyo.Var(within = pyo.NonNegativeReals)
model.x = pyo.Var(model.n, within = pyo.NonNegativeReals)

In [110]:
#objective
model.obj = pyo.Objective(expr = model.epsilon, sense = pyo.minimize)


In [111]:
#consstraint
def con1_rule(model):
    for h in range((pow(2, len(N))) - 1):
        result = 0
        for t in model.t:
            for i in delta[h]:
                if model.p[i, t] > 0:
                    k1 = landa
                else:
                    k1 = mu
                result += k1 * model.p[i, t]

            s = sum(model.p[j, t] for j in delta[h])
            if s > 0:
                k2 = landa
            else:
                k2 = mu

            result -= k2 * s
        condition = result - sum(model.x[i] for i in delta[h]) <= model.epsilon
        print(condition)
    return condition
model.con1 = pyo.Constraint(rule = con1_rule)

- x[1]  <=  epsilon
- x[2]  <=  epsilon
- x[3]  <=  epsilon
11.0 - (x[1] + x[2])  <=  epsilon
11.0 - (x[1] + x[3])  <=  epsilon
- (x[2] + x[3])  <=  epsilon
11.0 - (x[1] + x[2] + x[3])  <=  epsilon


In [112]:
def con2_rule(model):
    result = 0
    for h in range(1):
        for t in model.t:
            for i in N:
                if model.p[i,t]>0:
                    k1 = landa
                else:
                    k1 = mu
                result += k1*model.p[i,t]
            s = sum(model.p[j,t] for j in N)
            if s > 0:
                k2 = landa
            else:
                k2 = mu
            result -= k2*s
                
    condition = sum(model.x[i] for i in N) == result
    print(condition)
    return condition
    
model.con2 = pyo.Constraint(rule = con2_rule)    

    

x[1] + x[2] + x[3]  ==  11.0


In [113]:
opt = pyo.SolverFactory('glpk')
res = opt.solve(model)
model.display()

Model unknown

  Variables:
    epsilon : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    x : Size=3, Index=n
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :  11.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    con1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  0.0 :   0.0
    con2 : Size=1
        Key  : Lower : Body : Upper
        None :  11.0 : 11.0 :  11.0
